# remove the combined_mask.nii.gz files, if they are there. 

In [3]:

import os
import glob 
combined_scan = glob.glob('MRI_data/s*/*/combined*')
print(len(combined_scan ))
# remove these files
for file in combined_scan:
    os.remove(file)
combined_scan = glob.glob('MRI_data/s*/*/combined*')
print(len(combined_scan ))


0
0


# make a file name 'label_names.xlsx'

In [16]:
file_names = os.listdir('MRI_data/s0001/segmentations')
len(file_names)
# sort the file names
file_names.sort()
# save the file name in a excel file and with coumn name as 'file_name' and next column as 'label' and number 1 to till the end
import pandas as pd
df = pd.DataFrame(file_names, columns = ['label_names'])
df['multiplied_labels'] = range(1, len(file_names)+1)
df.to_excel('label_names.xlsx', index = False)

In [ ]:
import pandas as pd
import glob
import nibabel as nib
import numpy as np
import tqdm
import os 
import glob
 
combined_scan = glob.glob('MRI_data/s*/*/combined*')
print(len(combined_scan ))
# remove these files
for file in combined_scan:
    os.remove(file)
combined_scan = glob.glob('MRI_data/s*/*/combined*')
print('removed all combined scans, now scan (combined scans) numbers are: ', len(combined_scan ))


# Load label information from Excel
df = pd.read_excel('label_names.xlsx')
label_names = df['label_names'].tolist()
multiplied_labels = df['multiplied_labels'].tolist()
print(label_names[0], len(label_names))
print(multiplied_labels[0], len(multiplied_labels))

# Get list of folders containing masks
folders = glob.glob('MRI_data/s*/se*')
folders.sort()

for folder_num, folder in enumerate(tqdm.tqdm(folders, desc="Processing folders")):
    # Load one mask to get the shape and affine information
    initial_file = os.path.join(folder, label_names[0])
    initial_mask_nii = nib.load(initial_file)
    mask_shape = initial_mask_nii.get_fdata().shape
    affine = initial_mask_nii.affine

    # Initialize the combined mask with zeros
    combined_mask = np.zeros(mask_shape)
    non_empty_label_count = 0
    empty_masks = []

    for label_num, (label_name, label_value) in enumerate(tqdm.tqdm(zip(label_names, multiplied_labels))):
        file_path = os.path.join(folder, label_name)
        
        # Load current mask
        current_mask_nii = nib.load(file_path)
        current_mask = current_mask_nii.get_fdata()
        
        # Check if the current mask has any non-zero values
        if np.any(current_mask):
            non_empty_label_count += 1
        else:
            empty_masks.append(label_name)
            continue  # Skip processing empty masks

        # Scale the mask to assign unique label values
        current_mask_scaled = current_mask * label_value

        # Identify common regions where both masks have non-zero values
        common_regions = (current_mask > 0) & (combined_mask > 0)
        if np.any(common_regions):
            combined_mask[common_regions] = current_mask_scaled[common_regions]
            #print(f"Updated {np.sum(common_regions)} common regions for label '{label_name}' in folder '{folder}'")

        # Identify new regions to add to the combined mask where combined_mask is zero
        new_regions = (current_mask > 0) & (combined_mask == 0)
        combined_mask[new_regions] = current_mask_scaled[new_regions]

    # Exclude the background label from the count
    labels_in_mask = len(np.unique(combined_mask)) - 1  # Subtract 1 to exclude zero
    #print(f"{folder} Labels in mask: {labels_in_mask} Non-empty Labels: {non_empty_label_count}")

    # Save the combined mask
    combined_mask_nii = nib.Nifti1Image(combined_mask, affine)
    output_path = os.path.join(folder, 'combined_mask.nii.gz')
    nib.save(combined_mask_nii, output_path)


0
removed all combined scans, now scan (combined scans) numbers are:  0
adrenal_gland_left.nii.gz 56
1 56


56it [00:39,  1.43it/s]%|          | 0/298 [00:00<?, ?it/s]
56it [00:08,  6.83it/s]%|          | 1/298 [00:40<3:22:14, 40.86s/it]
56it [00:00, 115.11it/s]|          | 2/298 [00:49<1:48:02, 21.90s/it]
56it [00:01, 31.98it/s]%|          | 3/298 [00:50<59:49, 12.17s/it]  
56it [00:00, 84.76it/s]%|▏         | 4/298 [00:52<39:57,  8.15s/it]
56it [00:00, 107.22it/s]|▏         | 5/298 [00:52<26:43,  5.47s/it]
56it [00:09,  5.71it/s]%|▏         | 6/298 [00:53<18:30,  3.80s/it]
56it [00:02, 22.20it/s]%|▏         | 7/298 [01:03<28:33,  5.89s/it]
56it [00:01, 32.20it/s]%|▎         | 8/298 [01:06<23:28,  4.86s/it]
56it [01:01,  1.10s/it]%|▎         | 9/298 [01:08<18:51,  3.92s/it]
56it [00:15,  3.58it/s]%|▎         | 10/298 [02:12<1:48:50, 22.68s/it]
56it [00:05, 10.25it/s]%|▎         | 11/298 [02:28<1:39:06, 20.72s/it]
56it [00:00, 127.53it/s]|▍         | 12/298 [02:34<1:17:00, 16.16s/it]
56it [00:04, 12.81it/s]%|▍         | 13/298 [02:35<54:11, 11.41s/it]  
56it [00:09,  5.62it/s]%|▍         | 1

KeyboardInterrupt: 

# Devide, Train, val and Test (there is no test so I am making test to val)


In [30]:
import pandas as pd
df = pd.read_csv('meta.csv')
df1 = df[df['split']=='train']
print(len(df1))
df2 = df[df['split']=='val']
print(len(df2))
len(df1) + len(df2)
df['split'].value_counts()

268
0


split
train    268
test      30
Name: count, dtype: int64

In [26]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_csv('meta.csv')
# select only train values in split column
df = df[df['split']=='train']
print(df.columns)
# make directories for train and test
import os
os.makedirs('dataset_nii/train/images', exist_ok=True)
os.makedirs('dataset_nii/train/labels', exist_ok=True)


# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
folder = 'MRI_data' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = folder+'/'+files[i]+'/mri.nii.gz'
    labels = folder+'/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/train/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/train/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'manufacturer',
       'scanner_model', 'slice_thickness', 'scanning_sequence',
       'repetition_time', 'echo_time', 'magnetic_field_strength', 'source',
       'split'],
      dtype='object')


100%|██████████| 268/268 [00:02<00:00, 102.86it/s]


In [32]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_csv('meta.csv')
# select only train values in split column
df = df[df['split']=='test']
print(df.columns)
# make directories for train and test
import os

os.makedirs('dataset_nii/val/images', exist_ok=True)
os.makedirs('dataset_nii/val/labels', exist_ok=True)


# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
folder = 'MRI_data' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = folder+'/'+files[i]+'/mri.nii.gz'
    labels = folder+'/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/val/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/val/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'manufacturer',
       'scanner_model', 'slice_thickness', 'scanning_sequence',
       'repetition_time', 'echo_time', 'magnetic_field_strength', 'source',
       'split'],
      dtype='object')


100%|██████████| 30/30 [00:00<00:00, 105.45it/s]


# now convert the nii.gz images into tiff 

In [33]:
import m_yolo
from m_yolo import nii_rgb
import os
import nibabel as nib
import numpy as np
from PIL import Image
import tqdm
file = os.listdir('dataset_nii/train/images')
images = 'dataset_nii/train/images'
masks = 'dataset_nii/train/labels'
output_dir = 'dataset/train'
os.makedirs(output_dir, exist_ok=True)

for i in tqdm.tqdm(range(len(file))):
    image_path = os.path.join(images, file[i])
    mask_path = os.path.join(masks, file[i])
    nii_rgb.save_slices_as_rgb(image_path, mask_path, output_dir, name=file[i], angle=0)


100%|██████████| 268/268 [02:33<00:00,  1.75it/s]


In [35]:
import m_yolo
from m_yolo import nii_rgb
import os
import nibabel as nib
import numpy as np
from PIL import Image
import tqdm
file = os.listdir('dataset_nii/val/images')
images = 'dataset_nii/val/images'
masks = 'dataset_nii/val/labels'
output_dir = 'dataset/val'
os.makedirs(output_dir, exist_ok=True)

for i in tqdm.tqdm(range(len(file))):
    image_path = os.path.join(images, file[i])
    mask_path = os.path.join(masks, file[i])
    nii_rgb.save_slices_as_rgb(image_path, mask_path, output_dir, name=file[i], angle=0)


100%|██████████| 30/30 [00:20<00:00,  1.48it/s]


# tiff labels to .txt

In [36]:
class_mapping = {1:0, 2:1, 3: 2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9, 11:10, 12:11, 13:12, 14:13, 15:14, 16:15, 17:16, 18:17, 19:18, 20:19, 21:20, 22:21, 23:22, 24:23, 25:24, 26:25, 27:26, 28:27, 29:28, 30:29, 31:30, 32:31, 33:32, 34:33, 35:34, 36:35, 37:36, 38:37, 39:38, 40:39, 41:40, 42:41, 43:42, 44:43, 45:44, 46:45, 47:46, 48:47, 49:48, 50:49, 51:50, 52:51, 53:52, 54:53, 55:54, 56:55, 57:56, 58:57, 59:58, 60:59, 61:60, 62:61, 63:62, 64:63, 65:64, 66:65, 67:66, 68:67, 69:68, 70:69, 71:70, 72:71, 73:72, 74:73, 75:74, 76:75, 77:76, 78:77, 79:78, 80:79, 81:80, 82:81, 83:82, 84:83, 85:84, 86:85, 87:86, 88:87, 89:88, 90:89, 91:90,
                 92:91, 93:92,  94:93, 95:94, 96:95, 97:96, 98:97, 99:98, 100:99, 101:100, 102:101, 103:102, 104:103, 105:104, 106:105, 107:106, 108:107, 109:108, 110:109, 111:110, 112:111, 113:112, 114:113, 115:114, 116:115, 117:116, 118:117, 119:118, 120:119, 121:120, 122:121, 123:122, 124:123, 125:124, 126:125, 127:126, 128:127, 129:128, 130:129, 131:130, 132:131, 133:132, 134:133, 135:134, 136:135, 137:136, 138:137, 139:138, 140:139, 141:140, 142:141, 143:142, 144:143, 145:144, 146:145, 147:146, 148:147, 149:148, 150:149, 151:150, 152:151, 153:152, 154:153, 155:154, 156:155, 157:156, 158:157, 159:158, 160:159, 161:160, 162:161, 
                 163:162, 164:163, 165:164, 166:165, 167:166, 168:167, 169:168, 170:169, 171:170, 172:171, 173:172, 174:173, 175:174, 176:175, 177:176, 178:177, 179:178, 180:179, 181:180, 182:181, 183:182, 184:183, 185:184, 186:185, 187:186, 188:187, 189:188, 190:189, 191:190, 192:191, 193:192, 194:193, 195:194, 196:195, 197:196, 198:197, 199:198, 200:199, 201:200}

import m_yolo
from m_yolo import tiff_to_txt
import os 
import tqdm 
input_mask_path = 'dataset/train/masks/'
file = os.listdir(input_mask_path)
print('total number of Training masks: ', len(file))
for i in tqdm.tqdm(range(len(file))):
    input_mask_path = 'dataset/train/masks/' + file[i]
    output_dir = 'dataset/train/labels/'
    os.makedirs(output_dir, exist_ok=True)
    output_txt_path = output_dir + file[i].replace('.tiff', '.txt')
    tiff_to_txt.convert_mask_to_segmentation_(input_mask_path, output_txt_path, class_mapping)



input_mask_path = 'dataset/val/masks/'
file = os.listdir(input_mask_path)
print('total number of validation masks: ', len(file))
for i in tqdm.tqdm(range(len(file))):
    input_mask_path = 'dataset/val/masks/' + file[i]
    output_dir = 'dataset/val/labels/'
    os.makedirs(output_dir, exist_ok=True)
    output_txt_path = output_dir + file[i].replace('.tiff', '.txt')
    tiff_to_txt.convert_mask_to_segmentation_(input_mask_path, output_txt_path, class_mapping)



100%|██████████| 51663/51663 [01:33<00:00, 551.57it/s]


total number of validation masks:  6328


100%|██████████| 6328/6328 [00:11<00:00, 541.58it/s]


In [10]:
# please remove .DS_Store files from the dataset folder, and subfolders 
import os
import glob
import tqdm
folders = glob.glob('MRI_data/*/*/*')
folders


['MRI_data/s0290/segmentations/pancreas.nii.gz',
 'MRI_data/s0290/segmentations/lung_left.nii.gz',
 'MRI_data/s0290/segmentations/heart.nii.gz',
 'MRI_data/s0290/segmentations/esophagus.nii.gz',
 'MRI_data/s0290/segmentations/gluteus_maximus_right.nii.gz',
 'MRI_data/s0290/segmentations/gallbladder.nii.gz',
 'MRI_data/s0290/segmentations/kidney_left.nii.gz',
 'MRI_data/s0290/segmentations/iliac_vena_right.nii.gz',
 'MRI_data/s0290/segmentations/quadriceps_femoris_left.nii.gz',
 'MRI_data/s0290/segmentations/stomach.nii.gz',
 'MRI_data/s0290/segmentations/iliopsoas_right.nii.gz',
 'MRI_data/s0290/segmentations/gluteus_medius_right.nii.gz',
 'MRI_data/s0290/segmentations/spleen.nii.gz',
 'MRI_data/s0290/segmentations/humerus_right.nii.gz',
 'MRI_data/s0290/segmentations/tibia.nii.gz',
 'MRI_data/s0290/segmentations/femur_right.nii.gz',
 'MRI_data/s0290/segmentations/iliopsoas_left.nii.gz',
 'MRI_data/s0290/segmentations/sartorius_left.nii.gz',
 'MRI_data/s0290/segmentations/aorta.nii.gz'

In [ ]:


dataset_dir


In [12]:
from pathlib import Path
import pandas as pd
import os

def make_yaml(excel_relative_path="label_names.xlsx"):
    # Get the current working directory
    current_dir = Path(os.getcwd()).resolve()
    print(f"Current Working Directory: {current_dir}")

    # Search for a dataset folder recursively from the current directory
    dataset_dir = next(current_dir.rglob("dataset"), None)
    if not dataset_dir:
        raise FileNotFoundError("Dataset directory not found.")
    dataset_dir = dataset_dir.resolve()
    print(f"Dataset Directory Found: {dataset_dir}")

    train_path = (dataset_dir / "train/images").resolve()
    val_path = (dataset_dir / "val/images").resolve()
    test_path = (dataset_dir / "test/images").resolve()

    # Generate YAML content dynamically
    yaml_content = f'''
train: {train_path}
val: {val_path}
test: {test_path}

names:
'''

    excel_file_path = (current_dir / excel_relative_path).resolve()
    print(f"Looking for label_names.xlsx at: {excel_file_path}")
    if not excel_file_path.is_file():
        raise FileNotFoundError("label_names.xlsx file not found.")

    data = pd.read_excel(excel_file_path)
    print(f"Labels Found: {data['label_names'].tolist()}")

    # Adding label names dynamically from the DataFrame
    for i, name in enumerate(data['label_names']):
        yaml_content += f"    {i}: '{name.split('.')[0]}'\n"  # Strip file extension if present

    # Append hyperparameters (optional section for augmentation and training details)
    yaml_content += '''
# Hyperparameters ------------------------------------------------------------------------------------------------------
degrees: 0.5  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.2  # image scale (+/- gain)
shear: 0.2  # image shear (+/- deg) from -0.5 to 0.5
perspective: 0.1  # image perspective (+/- fraction), range 0-0.001
flipud: 0.7  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.8  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
'''

    # Save the YAML file
    yaml_path = current_dir / 'data.yaml'
    yaml_path.write_text(yaml_content)
    print(f"YAML file successfully created at: {yaml_path}")

    return yaml_path


In [13]:
make_yaml()

Current Working Directory: /home/sumitpandey/Documents/MED-YOLO/Total_segmentator_MRI
Dataset Directory Found: /home/sumitpandey/Documents/MED-YOLO/Total_segmentator_MRI/dataset
Looking for label_names.xlsx at: /home/sumitpandey/Documents/MED-YOLO/Total_segmentator_MRI/label_names.xlsx
Labels Found: ['adrenal_gland_left.nii.gz', 'adrenal_gland_right.nii.gz', 'aorta.nii.gz', 'autochthon_left.nii.gz', 'autochthon_right.nii.gz', 'brain.nii.gz', 'colon.nii.gz', 'duodenum.nii.gz', 'esophagus.nii.gz', 'femur_left.nii.gz', 'femur_right.nii.gz', 'fibula.nii.gz', 'gallbladder.nii.gz', 'gluteus_maximus_left.nii.gz', 'gluteus_maximus_right.nii.gz', 'gluteus_medius_left.nii.gz', 'gluteus_medius_right.nii.gz', 'gluteus_minimus_left.nii.gz', 'gluteus_minimus_right.nii.gz', 'heart.nii.gz', 'hip_left.nii.gz', 'hip_right.nii.gz', 'humerus_left.nii.gz', 'humerus_right.nii.gz', 'iliac_artery_left.nii.gz', 'iliac_artery_right.nii.gz', 'iliac_vena_left.nii.gz', 'iliac_vena_right.nii.gz', 'iliopsoas_left.ni

PosixPath('/home/sumitpandey/Documents/MED-YOLO/Total_segmentator_MRI/data.yaml')